# James spike time project

author: steeve.laquitaine@epfl.ch

### 1. Simulation description

Here is the simulation campaign we discussed: 

`/gpfs/bbp.cscs.ch/project/proj83/scratch/home/isbister/post_physiology_2023/sim_data/campaigns/spikesorting_stimulus_tests/8_spikesorting_stimulus_test_neuropixels_13-4-24/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0/12c5ef9a-1f3b-4b18-b3f5-ec6a417f645c`


* Only 12 simulations (indices 0 to 11) of the 72 simulations in that directory have so far been run (we might run more later).
* Each simulation is of 51.5 seconds of biological time so we have about 10 minutes of biological time from the 12 sims)
* Each simulation applies a whisker deflection at a particular angle 250 times at 200ms intervals starting at 1500ms (1500 + 250 * 200 = 51500ms)
* The stimulus rotations increment by 2 degrees. i.e. simulation 0 == 0 degrees, simulation 1 == 2 degrees, simulation 2 == 4 degrees etc.
* The extracellular recording is made at 40KHz, unlike the previous stimulus-evoked recording at 20KHz

### 2. Setup new experiment

1. Add data and output paths in `conf/dataset.yml` and run parameters in `conf/param.yml`.
2. Update `src/nodes/utils.py` with the new experiment's config.
3. Update `src/nodes/dataeng/lfp_only/campaign_stacking.py` with the new experiment's config.

### 3. (2h) Preprocess

```bash
sbatch cluster/processing/for_james/process.sbatch
#Submitted batch job 2131048 - done
```

### 4. (20m) Sort

```bash
sbatch cluster/sorting/others/for_james/sort_ks3.sbatch
#Submitted batch job 2138139
```


In [4]:
# SETUP PACKAGES 
%load_ext autoreload
%autoreload 2
import os
import spikeinterface as si

# SET PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config
from src.nodes.postpro.cell_matching import get_SpikeInterface_matching_object

# DATASETS

EXPERIMENT = "others/for_james"
SIMULATION_DATE = "2024_04_13"
data_cfg, _ = get_config(EXPERIMENT, SIMULATION_DATE).values()
TRUTH_PATH = data_cfg["sorting"]["simulation"]["ground_truth"]["output"]
SORTING_PATH = data_cfg["sorting"]["sorters"]["kilosort3"]["output"]
SCORES_PATH = data_cfg["postprocessing"]["agreement_scores"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-04-22 16:29:32,597 - root - utils.py - get_config - INFO - Reading experiment config.
2024-04-22 16:29:32,598 - root - utils.py - get_config_for_james - INFO - conf/others/for_james/2024_04_13 config
2024-04-22 16:29:32,618 - root - utils.py - get_config - INFO - Reading experiment config. - done


In [2]:
matching = get_SpikeInterface_matching_object(TRUTH_PATH, SORTING_PATH)
agreement_scores = matching.agreement_scores

In [17]:
import shutil
import pandas as pd

parent_folder = os.path.dirname(SCORES_PATH)
shutil.rmtree(parent_folder, ignore_errors=True)
os.makedirs(parent_folder)
agreement_scores.to_csv(SCORES_PATH)

# rows: true units
# cols: sorted units
print(SCORES_PATH)
pd.read_csv(SCORES_PATH)

/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/postpro/0_silico/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0/12c5ef9a-1f3b-4b18-b3f5-ec6a417f645c/ks3_agreement_scores/unit_matching.csv


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1453,1454,1455,1456,1457,1458,1459,1460,1461,1462
0,12165,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,16652,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,17923,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,18371,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,19690,0.000890,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,4223302,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1548,4225319,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1549,4228700,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1550,4229218,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [22]:
import spikeinterface as si

# sorted units
Sorting = si.load_extractor(SORTING_PATH)
print(Sorting.get_all_spike_trains())
print(Sorting.get_unit_ids())

# ground truth
SortingTrue = si.load_extractor(TRUTH_PATH)
print(SortingTrue.get_all_spike_trains())
print(SortingTrue.get_unit_ids())

/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0/12c5ef9a-1f3b-4b18-b3f5-ec6a417f645c/SortingKS3/
[(array([     259,      265,      267, ..., 24239343, 24239696, 24239824]), array([1112, 1013, 1117, ..., 1200,  535, 1017]))]
[   0    1    2 ... 1460 1461 1462]
/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0/12c5ef9a-1f3b-4b18-b3f5-ec6a417f645c/campaign/true_spikes
[(array([     193,      246,      249, ..., 24239416, 24239676, 24239820]), array([1808832, 3463535, 3712631, ..., 3474368, 3666861, 4064376]))]
[  12165   16652   17923 ... 4228700 4229218 4229506]


**Output to James**:

Sorted units path
/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0/12c5ef9a-1f3b-4b18-b3f5-ec6a417f645c/SortingKS3/

True units path
/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/sorting/0_silico/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0/12c5ef9a-1f3b-4b18-b3f5-ec6a417f645c/campaign/true_spikes

Unit matching:
/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/postpro/0_silico/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0/12c5ef9a-1f3b-4b18-b3f5-ec6a417f645c/ks3_agreement_scores/unit_matching.csv